In [2]:
D.<x> =PolynomialRing(QQ)
DD=PolynomialRing(QQ,'q')
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

def Delta(n):
    M = ModularForms(SL2Z, 12) 
    return DD(M.q_expansion_basis(n)[0])

def tau(n):
    return Delta(n+1)[n]
 
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})


print("okay")

okay


In [70]:
ls=[]
for n in [1..5]:ls=ls+[tau(n)]
print(ls)

[1, -24, 252, -1472, 4830]


In [71]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)

print(RamanujanCongruenceSeries(5))



-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [75]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)


def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

for k in [1..3]:
    print(RamanujanCongruenceSeries(n));print()
    print(recip(n)^k);print()
    rpc=recipPowerConstantTerm(k)
    test=order(2,rpc)-3*digitsum(2,k)
    print((rpc,test))
    
    print("----------------------------------------------------------")

-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-1 + 24 + 324*x + 3200*x^2 + 25650*x^3 + 176256*x^4 + 1056976*x^5 + O(x^6)

(24, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-2 + 48*x^-1 + 1224 + 21952*x + 309876*x^2 + 3657312*x^3 + 37435440*x^4 + O(x^5)

(1224, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-3 + 72*x^-2 + 2700*x^-1 + 70080 + 1412550*x + 23531040*x^2 + 336770016*x^3 + O(x^4)

(70080, 0)
----------------------------------------------------------


In [7]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..10]: # 100 < precision is necessary
        start=time.time()
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        ds=digitsum(3,a)
        test=o3-ds
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.0019698143005371094)
(1, 2, 0, 0.04666304588317871)
(1, 3, 0, 0.0288388729095459)
(1, 4, 0, 0.07622671127319336)
(1, 5, 0, 0.08355569839477539)
(1, 6, 0, 0.05843782424926758)
(1, 7, 0, 0.10085511207580566)
(1, 8, 0, 0.07990598678588867)
(1, 9, 0, 0.08585000038146973)
(1, 10, 0, 0.0899510383605957)

bad a's for trial  1

[]
------------------------------------------------------------------
no:

[(1, 0)]


In [8]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..100]: # 100 < precision is necessary
        print((b,a))
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        ds=digitsum(3,a)
        test=o3-ds
        tests=tests+[test]
    print("trial",b)
    print(tests)
    print("-----------------------------------------------------------------------")


(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(1, 34)
(1, 35)
(1, 36)
(1, 37)
(1, 38)
(1, 39)
(1, 40)
(1, 41)
(1, 42)
(1, 43)
(1, 44)
(1, 45)
(1, 46)
(1, 47)
(1, 48)
(1, 49)
(1, 50)
(1, 51)
(1, 52)
(1, 53)
(1, 54)
(1, 55)
(1, 56)
(1, 57)
(1, 58)
(1, 59)
(1, 60)
(1, 61)
(1, 62)
(1, 63)
(1, 64)
(1, 65)
(1, 66)
(1, 67)
(1, 68)
(1, 69)
(1, 70)
(1, 71)
(1, 72)
(1, 73)
(1, 74)
(1, 75)
(1, 76)
(1, 77)
(1, 78)
(1, 79)
(1, 80)
(1, 81)
(1, 82)
(1, 83)
(1, 84)
(1, 85)
(1, 86)
(1, 87)
(1, 88)
(1, 89)
(1, 90)
(1, 91)
(1, 92)
(1, 93)
(1, 94)
(1, 95)
(1, 96)
(1, 97)
(1, 98)
(1, 99)
(1, 100)
trial 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
import random
import time


def specialModulus(a):
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
tests=[]
for b in [1..10]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        c=recipPowerConstantTerm(a)
        o3=order(3,c)
        sk=c/3^o3
        m=mod(sk,3)
        if ((mod(a,2)==1)&(m!=2)):no=no+[a]
        if ((mod(a,2)==0)&(m!=1)):no=no+[a]
        ds=digitsum(3,a)
        test=o3-ds
        tests=tests+[test]
        print((b,a,m,len(no),test))
    print();print("trial",b)
    print();print(tests)
    print("-----------------------------------------------------------------------")


(1, 1, 2, 0, 0)
(1, 2, 1, 0, 0)
(1, 3, 2, 0, 0)
(1, 4, 1, 0, 0)
(1, 5, 2, 0, 0)
(1, 6, 1, 0, 0)
(1, 7, 2, 0, 0)
(1, 8, 1, 0, 0)
(1, 9, 2, 0, 0)
(1, 10, 1, 0, 0)
(1, 11, 2, 0, 0)
(1, 12, 1, 0, 0)
(1, 13, 2, 0, 0)
(1, 14, 1, 0, 0)
(1, 15, 2, 0, 0)
(1, 16, 1, 0, 0)
(1, 17, 2, 0, 0)
(1, 18, 1, 0, 0)
(1, 19, 2, 0, 0)
(1, 20, 1, 0, 0)
(1, 21, 2, 0, 0)
(1, 22, 1, 0, 0)
(1, 23, 2, 0, 0)
(1, 24, 1, 0, 0)
(1, 25, 2, 0, 0)
(1, 26, 1, 0, 0)
(1, 27, 2, 0, 0)
(1, 28, 1, 0, 0)
(1, 29, 2, 0, 0)
(1, 30, 1, 0, 0)
(1, 31, 2, 0, 0)
(1, 32, 1, 0, 0)
(1, 33, 2, 0, 0)
(1, 34, 1, 0, 0)
(1, 35, 2, 0, 0)
(1, 36, 1, 0, 0)
(1, 37, 2, 0, 0)
(1, 38, 1, 0, 0)
(1, 39, 2, 0, 0)
(1, 40, 1, 0, 0)
(1, 41, 2, 0, 0)
(1, 42, 1, 0, 0)
(1, 43, 2, 0, 0)
(1, 44, 1, 0, 0)
(1, 45, 2, 0, 0)
(1, 46, 1, 0, 0)
(1, 47, 2, 0, 0)
(1, 48, 1, 0, 0)
(1, 49, 2, 0, 0)
(1, 50, 1, 0, 0)
(1, 51, 2, 0, 0)
(1, 52, 1, 0, 0)
(1, 53, 2, 0, 0)
(1, 54, 1, 0, 0)
(1, 55, 2, 0, 0)
(1, 56, 1, 0, 0)
(1, 57, 2, 0, 0)
(1, 58, 1, 0, 0)
(1, 59, 2, 0, 0)
(1, 6

(1, 463, 2, 0, 0)
(1, 464, 1, 0, 0)
(1, 465, 2, 0, 0)
(1, 466, 1, 0, 0)
(1, 467, 2, 0, 0)
(1, 468, 1, 0, 0)
(1, 469, 2, 0, 0)
(1, 470, 1, 0, 0)
(1, 471, 2, 0, 0)
(1, 472, 1, 0, 0)
(1, 473, 2, 0, 0)
(1, 474, 1, 0, 0)
(1, 475, 2, 0, 0)
(1, 476, 1, 0, 0)
(1, 477, 2, 0, 0)
(1, 478, 1, 0, 0)
(1, 479, 2, 0, 0)
(1, 480, 1, 0, 0)
(1, 481, 2, 0, 0)
(1, 482, 1, 0, 0)
(1, 483, 2, 0, 0)
(1, 484, 1, 0, 0)
(1, 485, 2, 0, 0)
(1, 486, 1, 0, 0)
(1, 487, 2, 0, 0)
(1, 488, 1, 0, 0)
(1, 489, 2, 0, 0)
(1, 490, 1, 0, 0)
(1, 491, 2, 0, 0)
(1, 492, 1, 0, 0)
(1, 493, 2, 0, 0)
(1, 494, 1, 0, 0)
(1, 495, 2, 0, 0)
(1, 496, 1, 0, 0)
(1, 497, 2, 0, 0)
(1, 498, 1, 0, 0)
(1, 499, 2, 0, 0)
(1, 500, 1, 0, 0)

trial 1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(2, 337, 2, 0, 0)
(2, 338, 1, 0, 0)
(2, 339, 2, 0, 0)
(2, 340, 1, 0, 0)
(2, 341, 2, 0, 0)
(2, 342, 1, 0, 0)
(2, 343, 2, 0, 0)
(2, 344, 1, 0, 0)
(2, 345, 2, 0, 0)
(2, 346, 1, 0, 0)
(2, 347, 2, 0, 0)
(2, 348, 1, 0, 0)
(2, 349, 2, 0, 0)
(2, 350, 1, 0, 0)
(2, 351, 2, 0, 0)
(2, 352, 1, 0, 0)
(2, 353, 2, 0, 0)
(2, 354, 1, 0, 0)
(2, 355, 2, 0, 0)
(2, 356, 1, 0, 0)
(2, 357, 2, 0, 0)
(2, 358, 1, 0, 0)
(2, 359, 2, 0, 0)
(2, 360, 1, 0, 0)
(2, 361, 2, 0, 0)
(2, 362, 1, 0, 0)
(2, 363, 2, 0, 0)
(2, 364, 1, 0, 0)
(2, 365, 2, 0, 0)
(2, 366, 1, 0, 0)
(2, 367, 2, 0, 0)
(2, 368, 1, 0, 0)
(2, 369, 2, 0, 0)
(2, 370, 1, 0, 0)
(2, 371, 2, 0, 0)
(2, 372, 1, 0, 0)
(2, 373, 2, 0, 0)
(2, 374, 1, 0, 0)
(2, 375, 2, 0, 0)
(2, 376, 1, 0, 0)
(2, 377, 2, 0, 0)
(2, 378, 1, 0, 0)
(2, 379, 2, 0, 0)
(2, 380, 1, 0, 0)
(2, 381, 2, 0, 0)
(2, 382, 1, 0, 0)
(2, 383, 2, 0, 0)
(2, 384, 1, 0, 0)
(2, 385, 2, 0, 0)
(2, 386, 1, 0, 0)
(2, 387, 2, 0, 0)
(2, 388, 1, 0, 0)
(2, 389, 2, 0, 0)
(2, 390, 1, 0, 0)
(2, 391, 2, 0, 0)
(2, 392, 1

(3, 127, 2, 0, 0)
(3, 128, 1, 0, 0)
(3, 129, 2, 0, 0)
(3, 130, 1, 0, 0)
(3, 131, 2, 0, 0)
(3, 132, 1, 0, 0)
(3, 133, 2, 0, 0)
(3, 134, 1, 0, 0)
(3, 135, 2, 0, 0)
(3, 136, 1, 0, 0)
(3, 137, 2, 0, 0)
(3, 138, 1, 0, 0)
(3, 139, 2, 0, 0)
(3, 140, 1, 0, 0)
(3, 141, 2, 0, 0)
(3, 142, 1, 0, 0)
(3, 143, 2, 0, 0)
(3, 144, 1, 0, 0)
(3, 145, 2, 0, 0)
(3, 146, 1, 0, 0)
(3, 147, 2, 0, 0)
(3, 148, 1, 0, 0)
(3, 149, 2, 0, 0)
(3, 150, 1, 0, 0)
(3, 151, 2, 0, 0)
(3, 152, 1, 0, 0)
(3, 153, 2, 0, 0)
(3, 154, 1, 0, 0)
(3, 155, 2, 0, 0)
(3, 156, 1, 0, 0)
(3, 157, 2, 0, 0)
(3, 158, 1, 0, 0)
(3, 159, 2, 0, 0)
(3, 160, 1, 0, 0)
(3, 161, 2, 0, 0)
(3, 162, 1, 0, 0)
(3, 163, 2, 0, 0)
(3, 164, 1, 0, 0)
(3, 165, 2, 0, 0)
(3, 166, 1, 0, 0)
(3, 167, 2, 0, 0)
(3, 168, 1, 0, 0)
(3, 169, 2, 0, 0)
(3, 170, 1, 0, 0)
(3, 171, 2, 0, 0)
(3, 172, 1, 0, 0)
(3, 173, 2, 0, 0)
(3, 174, 1, 0, 0)
(3, 175, 2, 0, 0)
(3, 176, 1, 0, 0)
(3, 177, 2, 0, 0)
(3, 178, 1, 0, 0)
(3, 179, 2, 0, 0)
(3, 180, 1, 0, 0)
(3, 181, 2, 0, 0)
(3, 182, 1

(4, 5, 2, 0, 0)
(4, 6, 1, 0, 0)
(4, 7, 2, 0, 0)
(4, 8, 1, 0, 0)
(4, 9, 2, 0, 0)
(4, 10, 1, 0, 0)
(4, 11, 2, 0, 0)
(4, 12, 1, 0, 0)
(4, 13, 2, 0, 0)
(4, 14, 1, 0, 0)
(4, 15, 2, 0, 0)
(4, 16, 1, 0, 0)
(4, 17, 2, 0, 0)
(4, 18, 1, 0, 0)
(4, 19, 2, 0, 0)
(4, 20, 1, 0, 0)
(4, 21, 2, 0, 0)
(4, 22, 1, 0, 0)
(4, 23, 2, 0, 0)
(4, 24, 1, 0, 0)
(4, 25, 2, 0, 0)
(4, 26, 1, 0, 0)
(4, 27, 2, 0, 0)
(4, 28, 1, 0, 0)
(4, 29, 2, 0, 0)
(4, 30, 1, 0, 0)
(4, 31, 2, 0, 0)
(4, 32, 1, 0, 0)
(4, 33, 2, 0, 0)
(4, 34, 1, 0, 0)
(4, 35, 2, 0, 0)
(4, 36, 1, 0, 0)
(4, 37, 2, 0, 0)
(4, 38, 1, 0, 0)
(4, 39, 2, 0, 0)
(4, 40, 1, 0, 0)
(4, 41, 2, 0, 0)
(4, 42, 1, 0, 0)
(4, 43, 2, 0, 0)
(4, 44, 1, 0, 0)
(4, 45, 2, 0, 0)
(4, 46, 1, 0, 0)
(4, 47, 2, 0, 0)
(4, 48, 1, 0, 0)
(4, 49, 2, 0, 0)
(4, 50, 1, 0, 0)
(4, 51, 2, 0, 0)
(4, 52, 1, 0, 0)
(4, 53, 2, 0, 0)
(4, 54, 1, 0, 0)
(4, 55, 2, 0, 0)
(4, 56, 1, 0, 0)
(4, 57, 2, 0, 0)
(4, 58, 1, 0, 0)
(4, 59, 2, 0, 0)
(4, 60, 1, 0, 0)
(4, 61, 2, 0, 0)
(4, 62, 1, 0, 0)
(4, 63, 2, 0, 0)
(4

...

(8, 466, 1, 0, 0)
(8, 467, 2, 0, 0)
(8, 468, 1, 0, 0)
(8, 469, 2, 0, 0)
(8, 470, 1, 0, 0)
(8, 471, 2, 0, 0)
(8, 472, 1, 0, 0)
(8, 473, 2, 0, 0)
(8, 474, 1, 0, 0)
(8, 475, 2, 0, 0)
(8, 476, 1, 0, 0)
(8, 477, 2, 0, 0)
(8, 478, 1, 0, 0)
(8, 479, 2, 0, 0)
(8, 480, 1, 0, 0)
(8, 481, 2, 0, 0)
(8, 482, 1, 0, 0)
(8, 483, 2, 0, 0)
(8, 484, 1, 0, 0)
(8, 485, 2, 0, 0)
(8, 486, 1, 0, 0)
(8, 487, 2, 0, 0)
(8, 488, 1, 0, 0)
(8, 489, 2, 0, 0)
(8, 490, 1, 0, 0)
(8, 491, 2, 0, 0)
(8, 492, 1, 0, 0)
(8, 493, 2, 0, 0)
(8, 494, 1, 0, 0)
(8, 495, 2, 0, 0)
(8, 496, 1, 0, 0)
(8, 497, 2, 0, 0)
(8, 498, 1, 0, 0)
(8, 499, 2, 0, 0)
(8, 500, 1, 0, 0)

trial 8

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(9, 5, 2, 0, 0)
(9, 6, 1, 0, 0)
(9, 7, 2, 0, 0)
(9, 8, 1, 0, 0)
(9, 9, 2, 0, 0)
(9, 10, 1, 0, 0)
(9, 11, 2, 0, 0)
(9, 12, 1, 0, 0)
(9, 13, 2, 0, 0)
(9, 14, 1, 0, 0)
(9, 15, 2, 0, 0)
(9, 16, 1, 0, 0)
(9, 17, 2, 0, 0)
(9, 18, 1, 0, 0)
(9, 19, 2, 0, 0)
(9, 20, 1, 0, 0)
(9, 21, 2, 0, 0)
(9, 22, 1, 0, 0)
(9, 23, 2, 0, 0)
(9, 24, 1, 0, 0)
(9, 25, 2, 0, 0)
(9, 26, 1, 0, 0)
(9, 27, 2, 0, 0)
(9, 28, 1, 0, 0)
(9, 29, 2, 0, 0)
(9, 30, 1, 0, 0)
(9, 31, 2, 0, 0)
(9, 32, 1, 0, 0)
(9, 33, 2, 0, 0)
(9, 34, 1, 0, 0)
(9, 35, 2, 0, 0)
(9, 36, 1, 0, 0)
(9, 37, 2, 0, 0)
(9, 38, 1, 0, 0)
(9, 39, 2, 0, 0)
(9, 40, 1, 0, 0)
(9, 41, 2, 0, 0)
(9, 42, 1, 0, 0)
(9, 43, 2, 0, 0)
(9, 44, 1, 0, 0)
(9, 45, 2, 0, 0)
(9, 46, 1, 0, 0)
(9, 47, 2, 0, 0)
(9, 48, 1, 0, 0)
(9, 49, 2, 0, 0)
(9, 50, 1, 0, 0)
(9, 51, 2, 0, 0)
(9, 52, 1, 0, 0)
(9, 53, 2, 0, 0)
(9, 54, 1, 0, 0)
(9, 55, 2, 0, 0)
(9, 56, 1, 0, 0)
(9, 57, 2, 0, 0)
(9, 58, 1, 0, 0)
(9, 59, 2, 0, 0)
(9, 60, 1, 0, 0)
(9, 61, 2, 0, 0)
(9, 62, 1, 0, 0)
(9, 63, 2, 0, 0)
(9

(9, 466, 1, 0, 0)
(9, 467, 2, 0, 0)
(9, 468, 1, 0, 0)
(9, 469, 2, 0, 0)
(9, 470, 1, 0, 0)
(9, 471, 2, 0, 0)
(9, 472, 1, 0, 0)
(9, 473, 2, 0, 0)
(9, 474, 1, 0, 0)
(9, 475, 2, 0, 0)
(9, 476, 1, 0, 0)
(9, 477, 2, 0, 0)
(9, 478, 1, 0, 0)
(9, 479, 2, 0, 0)
(9, 480, 1, 0, 0)
(9, 481, 2, 0, 0)
(9, 482, 1, 0, 0)
(9, 483, 2, 0, 0)
(9, 484, 1, 0, 0)
(9, 485, 2, 0, 0)
(9, 486, 1, 0, 0)
(9, 487, 2, 0, 0)
(9, 488, 1, 0, 0)
(9, 489, 2, 0, 0)
(9, 490, 1, 0, 0)
(9, 491, 2, 0, 0)
(9, 492, 1, 0, 0)
(9, 493, 2, 0, 0)
(9, 494, 1, 0, 0)
(9, 495, 2, 0, 0)
(9, 496, 1, 0, 0)
(9, 497, 2, 0, 0)
(9, 498, 1, 0, 0)
(9, 499, 2, 0, 0)
(9, 500, 1, 0, 0)

trial 9

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(10, 5, 2, 0, 0)
(10, 6, 1, 0, 0)
(10, 7, 2, 0, 0)
(10, 8, 1, 0, 0)
(10, 9, 2, 0, 0)
(10, 10, 1, 0, 0)
(10, 11, 2, 0, 0)
(10, 12, 1, 0, 0)
(10, 13, 2, 0, 0)
(10, 14, 1, 0, 0)
(10, 15, 2, 0, 0)
(10, 16, 1, 0, 0)
(10, 17, 2, 0, 0)
(10, 18, 1, 0, 0)
(10, 19, 2, 0, 0)
(10, 20, 1, 0, 0)
(10, 21, 2, 0, 0)
(10, 22, 1, 0, 0)
(10, 23, 2, 0, 0)
(10, 24, 1, 0, 0)
(10, 25, 2, 0, 0)
(10, 26, 1, 0, 0)
(10, 27, 2, 0, 0)
(10, 28, 1, 0, 0)
(10, 29, 2, 0, 0)
(10, 30, 1, 0, 0)
(10, 31, 2, 0, 0)
(10, 32, 1, 0, 0)
(10, 33, 2, 0, 0)
(10, 34, 1, 0, 0)
(10, 35, 2, 0, 0)
(10, 36, 1, 0, 0)
(10, 37, 2, 0, 0)
(10, 38, 1, 0, 0)
(10, 39, 2, 0, 0)
(10, 40, 1, 0, 0)
(10, 41, 2, 0, 0)
(10, 42, 1, 0, 0)
(10, 43, 2, 0, 0)
(10, 44, 1, 0, 0)
(10, 45, 2, 0, 0)
(10, 46, 1, 0, 0)
(10, 47, 2, 0, 0)
(10, 48, 1, 0, 0)
(10, 49, 2, 0, 0)
(10, 50, 1, 0, 0)
(10, 51, 2, 0, 0)
(10, 52, 1, 0, 0)
(10, 53, 2, 0, 0)
(10, 54, 1, 0, 0)
(10, 55, 2, 0, 0)
(10, 56, 1, 0, 0)
(10, 57, 2, 0, 0)
(10, 58, 1, 0, 0)
(10, 59, 2, 0, 0)
(10, 60, 1, 0, 

(10, 442, 1, 0, 0)
(10, 443, 2, 0, 0)
(10, 444, 1, 0, 0)
(10, 445, 2, 0, 0)
(10, 446, 1, 0, 0)
(10, 447, 2, 0, 0)
(10, 448, 1, 0, 0)
(10, 449, 2, 0, 0)
(10, 450, 1, 0, 0)
(10, 451, 2, 0, 0)
(10, 452, 1, 0, 0)
(10, 453, 2, 0, 0)
(10, 454, 1, 0, 0)
(10, 455, 2, 0, 0)
(10, 456, 1, 0, 0)
(10, 457, 2, 0, 0)
(10, 458, 1, 0, 0)
(10, 459, 2, 0, 0)
(10, 460, 1, 0, 0)
(10, 461, 2, 0, 0)
(10, 462, 1, 0, 0)
(10, 463, 2, 0, 0)
(10, 464, 1, 0, 0)
(10, 465, 2, 0, 0)
(10, 466, 1, 0, 0)
(10, 467, 2, 0, 0)
(10, 468, 1, 0, 0)
(10, 469, 2, 0, 0)
(10, 470, 1, 0, 0)
(10, 471, 2, 0, 0)
(10, 472, 1, 0, 0)
(10, 473, 2, 0, 0)
(10, 474, 1, 0, 0)
(10, 475, 2, 0, 0)
(10, 476, 1, 0, 0)
(10, 477, 2, 0, 0)
(10, 478, 1, 0, 0)
(10, 479, 2, 0, 0)
(10, 480, 1, 0, 0)
(10, 481, 2, 0, 0)
(10, 482, 1, 0, 0)
(10, 483, 2, 0, 0)
(10, 484, 1, 0, 0)
(10, 485, 2, 0, 0)
(10, 486, 1, 0, 0)
(10, 487, 2, 0, 0)
(10, 488, 1, 0, 0)
(10, 489, 2, 0, 0)
(10, 490, 1, 0, 0)
(10, 491, 2, 0, 0)
(10, 492, 1, 0, 0)
(10, 493, 2, 0, 0)
(10, 494, 1,